# Using `geom_imshow()` to draw DEM on map

[SRTM1N00E012V3](https://earthexplorer.usgs.gov/scene/metadata/full/5e83a3ee1af480c5/SRTM1N00E012V3/) courtesy of the U.S. Geological Survey

In [ ]:
import numpy as np
from PIL import Image
from osgeo import gdal

from lets_plot import *
from lets_plot.geo_data import *
LetsPlot.setup_html()

In [ ]:
plot_size = 600
subplot_size = 500

In [ ]:
# Data from geocoding
states = geocode_states().scope("Gabon").inc_res(4).get_boundaries()
cities = geocode_cities().scope("Gabon").get_centroids()

In [ ]:
# Data from DEM
dataset = gdal.Open("ivindo/river.bil")
dataset_array = dataset.ReadAsArray()
transform = dataset.GetGeoTransform()
lon_min = transform[0]
lon_max = lon_min + (transform[1] * dataset_array.shape[1])
lat_max = transform[3]
lat_min = lat_max + (transform[5] * dataset_array.shape[0])

In [ ]:
# Data from image
image_array = np.asarray(Image.open("osm_ivindo_screen.png"))

In [ ]:
plot = ggplot() + \
    geom_imshow(dataset_array, extent=[lon_min, lon_max, lat_min, lat_max], cmap="viridis") + \
    geom_map(data=states, color="black", alpha=0) + \
    geom_point(data=cities, shape=21, size=3, color="black", fill="white", \
               tooltips=layer_tooltips().line("@{found name}")) + \
    ggsize(plot_size, plot_size) + \
    theme_classic() + theme(axis="blank")
plot + ggtitle("Ivindo river in Gabon")

In [ ]:
plot += coord_map(xlim=[lon_min, lon_max], ylim=[lat_min, lat_max]) + \
    ggtitle("Plot")
image = ggplot() + \
    geom_imshow(image_array) + \
    ggsize(plot_size, plot_size) + \
    ggtitle("Image") + \
    theme_classic() + theme(axis="blank")

bunch = GGBunch()
bunch.add_plot(plot, 0, 0, subplot_size, subplot_size)
bunch.add_plot(image, subplot_size, 0, subplot_size, subplot_size)
bunch.show()